In [ ]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "chicago_crime" dataset
dataset_ref = client.dataset("chicago_crime", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

In [ ]:
# List all the tables in the "hacker_news" dataset
tables = list(client.list_tables(dataset))

# Print names of all tables in the dataset (there are four!)
for table in tables:
    print(table.table_id)

In [ ]:
table_ref = dataset_ref.table("full")

# API request - fetch the table
table = client.get_table(table_ref)

In [ ]:
# Preview the first five lines of the "full" table
client.list_rows(table, max_results=5).to_dataframe()

In [ ]:
# Query to select countries with units of "ppm"
first_query = """
        SELECT country
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE unit = 'ppm'
        """

# Set up the query (cancel the query if it would use too much of
# your quota, with the limit set to 10 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
first_query_job = client.query(first_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
first_results = first_query_job.to_dataframe()

# View top few rows of results
print(first_results.head())

# Check your answer
q_1.check()

In [ ]:
prolific_commenters_query = """
        SELECT author, COUNT(id) AS NumPosts
        FROM `bigquery-public-data.hacker_news.comments`
        GROUP BY author
        HAVING NumPosts > 10000
        """

In [ ]:
country_spend_pct_query = """
                          SELECT country_name, AVG(value) as avg_ed_spending_pct
                          FROM `bigquery-public-data.world_bank_intl_education.international_education`
                          WHERE indicator_code="SE.XPD.TOTL.GD.ZS" and year<=2017 and year>=2010
                          GROUP BY country_name
                          ORDER BY avg_ed_spending_pct DESC
                          """

In [ ]:
code_count_query = """
                          SELECT indicator_code,indicator_name, COUNT(*) as num_rows
                          FROM `bigquery-public-data.world_bank_intl_education.international_education`
                          WHERE year=2016
                          GROUP BY indicator_code,indicator_name
                          HAVING num_rows>=175
                          ORDER BY num_rows DESC
                          """

In [ ]:
speeds_query = """
               WITH RelevantRides AS
               (
                   SELECT *
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE trip_seconds>0 and trip_miles > 0 and trip_start_timestamp > "2017-01-01" and trip_start_timestamp < "2017-07-01"
               )
               SELECT EXTRACT(HOUR FROM trip_start_timestamp) as hour_of_day, COUNT(*) as num_trips,3600 * SUM(trip_miles) / SUM(trip_seconds) as avg_mph
               FROM RelevantRides
               GROUP BY hour_of_day
               ORDER BY hour_of_day
               """

In [ ]:
query = """
        SELECT L.license, COUNT(1) AS number_of_files
        FROM `bigquery-public-data.github_repos.sample_files` AS sf
        INNER JOIN `bigquery-public-data.github_repos.licenses` AS L
            ON sf.repo_name = L.repo_name
        GROUP BY L.license
        ORDER BY number_of_files DESC
        """

In [ ]:
questions_query = """
                  SELECT id,title,owner_user_id
                  FROM `bigquery-public-data.stackoverflow.posts_questions`
                  WHERE tags LIKE '%bigquery%'
                  """

In [ ]:
answers_query = """
        SELECT p.id, p.body, p.owner_user_id
        FROM `bigquery-public-data.stackoverflow.posts_answers` as p
        INNER JOIN `bigquery-public-data.stackoverflow.posts_questions` as o
            ON p.parent_id = o.id
        WHERE o.tags LIKE '%bigquery%'
        """